In [1]:
import torch
from torch import nn
from transformers import MBartModel, MBartTokenizer, MBartConfig
from transformers import AutoTokenizer
import multiprocessing
from datasets import concatenate_datasets, load_dataset
from tqdm import tqdm
from torch.utils import data

In [2]:
def model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024 ** 2
    return size_all_mb


def batch_iterator(dataset, batch_size=10000):
    for i in tqdm(range(0, len(dataset), batch_size)):
        yield dataset[i: i + batch_size]


def group_texts(examples, **kwargs):
    tokenizer = kwargs['0']
    tokenized_inputs = tokenizer(
        examples, return_special_tokens_mask=True, truncation=True, max_length=512
    )
    return tokenized_inputs

In [4]:
dataset_europarl = load_dataset("g8a9/europarl_en-it", split='train')
dataset_europarl = dataset_europarl.remove_columns('Unnamed: 0')

tokenizer = MBartTokenizer.from_pretrained("nikodallanoce/mbart-europarl-en-it")

num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

# preprocess dataset
tok_en = dataset_europarl.map(group_texts, batched=True, num_proc=8, input_columns=['sent_en'],
                              fn_kwargs={'0': tokenizer})

Using custom data configuration g8a9--europarl_en-it-e5b76d871c2cce3d
Found cached dataset csv (C:/Users/dllni/.cache/huggingface/datasets/g8a9___csv/g8a9--europarl_en-it-e5b76d871c2cce3d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Parameter 'indices'=range(0, 213726) of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


The max length for the tokenizer is: 1024
        

In [14]:
tok_en.set_format(type='numpy', columns=['input_ids', 'special_tokens_mask', 'attention_mask'])

te = tok_en['input_ids']
ids_att = data.TensorDataset(torch.from_numpy(tok_en['input_ids']))
#dataloader = torch.utils.data.DataLoader(ids_att, batch_size=64, drop_last=True, pin_memory_device='cuda:0', pin_memory=True)
#tok_en.reset_format()

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [18]:
from MBart import MBart
mbart_config = MBartConfig(encoder_layers=6, decoder_layers=6,
                               encoder_ffn_dim=512, decoder_ffn_dim=512,
                               encoder_attention_heads=8, decoder_attention_heads=8,
                               d_model=512)

model: nn.Module = MBart(mbart_config)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [20]:
from transformers import DataCollatorForLanguageModeling

ris = DataCollatorForLanguageModeling(tokenizer)(dataset_europarl['sent_en'])

AttributeError: 'str' object has no attribute 'size'